<a href="https://colab.research.google.com/github/YosefAshraf24/Face-Recognition/blob/main/Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages

In [ ]:
import os
import math
import hashlib
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg as la
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA as SKLPCA

# Load ORL Dataset

## Load Data and Labels

In [ ]:
def load_data(path):
  """
  Loads a dataset from a given path.

  Args:
    path: Path to the dataset

  Returns:
    data: A matrix of shape (num_samples, num_features) containing the data
    labels: A vector of shape (num_samples) containing the label for each sample
  """
  # Initialize data matrix and labels vector
  data = np.empty((0, 10304), dtype=np.uint8)
  labels = np.empty((0), dtype=np.uint8)
  
  # Get all possible labels from the folder names
  # A folder called "s1" corresponds to label 1, "s2" to label 2, etc.
  unique_labels = os.listdir(path)
  unique_labels = map(lambda x: int(x[1:]), unique_labels)
  unique_labels = sorted(unique_labels)
  unique_labels = np.array(unique_labels)

  # Loop through all labels
  for label in unique_labels:
    # Loop through all images labeled with the current label
    label_path = os.path.join(path, "s" + str(label))
    image_names = os.listdir(label_path)
    image_names = sorted(image_names, key=lambda x: int(x[:-4]))
    for image_name in image_names:
      # Read in the image and convert to grayscale
      image_path = os.path.join(label_path, image_name)
      image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)

      # Downscale the image to 112x92 and flatten it into a vector
      image = cv.resize(image, (112, 92))
      image = image.flatten()

      # Add collected data
      data = np.append(data, np.array([image]), axis=0)
      labels = np.append(labels, np.array([label]), axis=0)

  # Print some information about the dataset
  print(f"Dataset: {path}")
  print(f"  - Number of samples: {data.shape[0]}")
  print(f"  - Number of features: {data.shape[1]}")
  print(f"  - Number of unique labels: {unique_labels.shape[0]}")
  print()

  return data, labels

In [ ]:
data, labels = load_data('./archive')

## Split the Dataset into Training and Testing Sets

In [ ]:
# Split the data into training and testing sets with a 1:1 ratio
training_data = data[1::2]
training_labels = labels[1::2]
testing_data = data[::2]
testing_labels = labels[::2]

## Plot a Random Sample From the Dataset

In [ ]:
def plot_images_samples(data, labels, num_samples=5):
  """
  Plots a random sample of images from a dataset.

  Args:
    data: A matrix of shape (num_samples, num_features) containing the data
    labels: A vector of shape (num_samples) containing the label for each sample
    num_samples: The number of images to plot for each label (default: 5)

  Returns:
    None
  """
  # Get the unique labels and their cound
  unique_labels = np.unique(labels)
  num_labels = len(unique_labels)

  # Initialize the figure where we will plot the images
  _, axs = plt.subplots(num_labels, num_samples, figsize=(num_samples, num_labels))

  # Loop through all available labels
  for i, label in enumerate(unique_labels):
    # Choose a random sample of images for the current label
    label_indices = np.where(labels == label)[0]
    label_indices = np.random.choice(label_indices, num_samples, replace=False)

    # Loop through all images in the sample
    for j, index in enumerate(label_indices):
      # Plot the image
      image = data[index].reshape((92, 112))
      axs[i, j].imshow(image, cmap='gray')
      axs[i, j].axis('off')
      axs[i, j].set_title(label)
  plt.show()

In [ ]:
print("A sample of images from the training set:")
plot_images_samples(training_data, training_labels)

print("A sample of images from the testing set:")
plot_images_samples(testing_data, testing_labels)

# Classification Using PCA

## Algorithm Implementation

In [ ]:
def pca(data, alphas=[0.8, 0.85, 0.9, 0.95]):
  """
  Performs PCA on the given dataset.

  Args:
    data: A matrix of shape (num_samples, num_features) containing the data
    alphas: A list of floats between 0 and 1 indicating the variance to be explained
        by the PCA projection (default: [0.8, 0.85, 0.9, 0.95])

  Returns:
    proj_matrices: A list of projection matrices, one for each alpha
  """
  # Initialize the list of projection matrices
  proj_matrices = [None] * len(alphas)

  # Initialize eigenvectors and eigenvalues
  eig_vecs = None
  eig_vals = None

  # Get sha1 hash of the data
  hash = hashlib.sha1()
  hash.update(data.tobytes())
  hash = hash.hexdigest()

  # Check if the eigenvectors are cached
  if os.path.exists(f"./.cache/pca_{hash}.npz"):
    container = np.load(f"./.cache/pca_{hash}.npz")
    eig_vecs = container['arr_0']
    eig_vals = container['arr_1']
    del container
  else:
    # Compute the overall mean
    data_mean = np.mean(data, axis=0)

    # Center the data
    data_centered = data - data_mean

    # Compute the covariance matrix
    cov_mat = np.cov(data_centered, rowvar=False)

    # Calculate the eigenvectors and eigenvalues
    eig_vals, eig_vecs = la.eigh(cov_mat)

    # Sort the eigenvectors and eigenvalues in descending order
    idx = eig_vals.argsort()[::-1]
    eig_vals = eig_vals[idx]
    eig_vecs = eig_vecs[:, idx]

    # Cache the eigenvectors and eigenvalues
    if not os.path.exists("./.cache"):
        os.makedirs("./.cache")
    np.savez(f"./.cache/pca_{hash}.npz", eig_vecs, eig_vals)

  # Compute the cumulative explained variance
  cumulative_explained_variance = np.cumsum(eig_vals) / np.sum(eig_vals)
  
  # Find the least number of eigenvectors needed to explain the variance for each alpha
  for i, alpha in enumerate(alphas):
    num_eig = np.argmax(cumulative_explained_variance >= alpha) + 1
    proj_matrices[i] = eig_vecs[:, :num_eig]
  return proj_matrices

## Run the Algorithm

In [ ]:
# Initialize the list of alpha values
alphas = [0.8, 0.85, 0.9, 0.95]

# Compute the projection matrices for each alpha
proj_matrices = pca(training_data, alphas)

## Evaluate the Algorithm

In [ ]:
def plot_failed_predictions(data, labels, predictions):
    """
    Plots the images that are wrongly classified by a classifier.

    An example of each wrong class is also plotted.

    Args:
        data: A matrix of shape (num_samples, num_features) containing the data
            whose labels was predicted by the classifier.
        labels: A vector of shape (num_samples,) containing the true labels of the data.
        predictions: A vector of shape (num_samples,) containing the predicted labels of the data.

    Returns:
        None
    """
    # Find the indices of the failed predictions
    failed_predictions = np.where(predictions != labels)[0]

    # Find the predicted classes of the failed predictions
    wrong_classes = predictions[failed_predictions]

    # Find the images that are wrongly classified
    missed_images = data[failed_predictions]

    # Get an example of each wrong class
    wrong_images = [data[np.where(testing_labels == i)[0][0]] for i in wrong_classes]

    # Plot the images in two columns
    fig, axs = plt.subplots(len(failed_predictions), 2, figsize=(6, len(failed_predictions) * 2.5))
    fig.suptitle(f"{len(failed_predictions)}/{len(predictions)} Failed Predictions")
    fig.subplots_adjust(top=0.94)
    for i in range(len(failed_predictions)):
        axs[i, 0].set_title(f"Right class: {labels[failed_predictions[i]]}")
        axs[i, 0].imshow(missed_images[i].reshape(92, 112), cmap='gray')
        axs[i, 1].set_title(f"Predicted class: {wrong_classes[i]}")
        axs[i, 1].imshow(wrong_images[i].reshape(92, 112), cmap='gray')

    # turn off the axis
    for ax in axs.flat:
        ax.axis('off')
    plt.show()

In [ ]:
def plot_failed_xy_predictions(data, labels, predictions):
    """
    Plots the images that are wrongly classified by a T/F classifier.

    Args:
        data: A matrix of shape (num_samples, num_features) containing the data
            whose labels was predicted by the classifier.
        labels: A vector of shape (num_samples,) containing the true labels of the data.
        predictions: A vector of shape (num_samples,) containing the predicted labels of the data.

    Returns:
        None
    """
    # Find the indices of the failed predictions
    failed_predictions = np.where(predictions != labels)[0]

    # Find the images that are wrongly classified
    missed_images = data[failed_predictions]

    # Plot the images
    fig, axs = plt.subplots(math.ceil(len(failed_predictions) / 4), 4, figsize=(10, math.ceil(len(failed_predictions) / 4) * 2))
    fig.suptitle(f"{len(failed_predictions)}/{len(predictions)} Failed Predictions")
    fig.subplots_adjust(top=0.94)
    row = 0
    col = 0
    for i in range(len(failed_predictions)):
        axs[row, col].imshow(missed_images[i].reshape(92, 112), cmap='gray')
        col = (col + 1) % 4
        row = row + 1 if col == 0 else row

    # turn off the axis
    for ax in axs.flat:
        ax.axis('off')
    plt.show()

In [ ]:
def plot_success_xy_predictions(data, labels, predictions):
    """
    Plots the images that are correctly classified by a T/F classifier.

    Args:
        data: A matrix of shape (num_samples, num_features) containing the data
            whose labels was predicted by the classifier.
        labels: A vector of shape (num_samples,) containing the true labels of the data.
        predictions: A vector of shape (num_samples,) containing the predicted labels of the data.

    Returns:
        None
    """
    # Find the indices of the failed predictions
    correct_predictions = np.where(predictions == labels)[0]

    # Find the images that are wrongly classified
    found_images = testing_data[correct_predictions]

    # Plot the images
    fig, axs = plt.subplots(math.ceil(len(correct_predictions) / 20), 20, figsize=(10, math.ceil(len(correct_predictions) / 20) * 0.5))
    fig.suptitle(f"{len(correct_predictions)}/{len(predictions)} Correct Predictions")
    fig.subplots_adjust(top=0.9)
    row = 0
    col = 0
    for i in range(len(correct_predictions)):
        axs[row, col].imshow(found_images[i].reshape(92, 112), cmap='gray')
        col = (col + 1) % 20
        row = row + 1 if col == 0 else row

    # turn off the axis
    for ax in axs.flat:
        ax.axis('off')
    plt.show()

In [ ]:
def evaluate_model(
    proj_matrix,
    training_data,
    training_labels,
    testing_data,
    testing_labels,
    ks=[1, 3, 5, 7]
):
  """"
  Evaluates the performance of a k-NN classifier on the given data.

  Args:
      training_data: A matrix of shape (num_training_samples, num_features) containing the training data.
      training_labels: A vector of shape (num_training_samples,) containing the training labels.
      testing_data: A matrix of shape (num_testing_samples, num_features) containing the testing data.
      testing_labels: A vector of shape (num_testing_samples,) containing the testing labels.
      ks: A list of k values to use for the k-NN classifier.

  Returns:
      best: The best accuracy achieved by the k-NN classifier.
  """
  # Initialize the best accuracy and the list of accuracies for each k
  best = 0
  res = [None] * len(ks)

  # Check if classification is binary
  is_xy = np.unique(training_labels).shape[0] == 2

  # Project the data onto the subspace spanned by the projection matrix
  proj_trainning_data = np.dot(training_data, proj_matrix)
  proj_testing_data = np.dot(testing_data, proj_matrix)

  # Loop over each k
  for i, k in enumerate(ks):
    # Train the classifier and predict the labels
    neigh = KNeighborsClassifier(n_neighbors=k).fit(proj_trainning_data, training_labels)
    predictions = neigh.predict(proj_testing_data)

    # Compute the accuracy and update the best accuracy
    res[i] = np.mean(predictions == testing_labels)
    best = max(best, res[i])
    print(f"@nn = {k}, accuracy = {res[i]}")

    # Plot the images that are correctly and incorrectly classified if the
    # classification is binary. Otherwise, plot the incorrectly classified images only.
    if is_xy:
      plot_failed_xy_predictions(testing_data, testing_labels, predictions)
      plot_success_xy_predictions(testing_data, testing_labels, predictions)
    else:
      plot_failed_predictions(testing_data, testing_labels, predictions)

  # Plot the relationship between k and accuracy
  plt.plot(ks, res)
  plt.title('Accuracy of KNN classifier with LDA')
  plt.gcf().set_size_inches(15,5)
  plt.xlabel('K')
  plt.ylabel('accuracy')
  plt.show()
  return best

In [ ]:
# Evaluate the performance of the k-NN classifier for each alpha
for i in range(len(alphas)):
  print(f"Evaluating model with alpha = {alphas[i]}:")
  evaluate_model(proj_matrices[i], training_data, training_labels, testing_data, testing_labels)

# Classification Using LDA

## Algorithm Implementation

In [ ]:
def lda(data, labels, eigc=None):
    """"
    Performs Linear Discriminant Analysis on the given data.

    Args:
        data: A matrix of shape (num_samples, num_features) containing the data.
        labels: A vector of shape (num_samples,) containing the labels of the data.
        eigc: The number of eigenvectors to use for the projection matrix.

    Returns:
        proj_matrix: A matrix of shape (num_features, num_features) containing the projection matrix.
    """

    # Group data by class
    _, num_labels = np.unique(labels, return_counts=True)
    labels_idx = np.split(np.argsort(labels), np.cumsum(num_labels)[:-1])
    data_class = [data[idx] for idx in labels_idx]
    data = data[np.argsort(labels)]

    # Initialize eigenvectors variable
    eig_vecs = None

    # Get sha1 hash of the data and labels
    hash = hashlib.sha1()
    hash.update(data.tobytes())
    hash.update(labels.tobytes())
    hash = hash.hexdigest()

    # Check if the eigenvectors are cached
    if os.path.exists(f"./.cache/lda_{hash}.npy"):
        eig_vecs = np.load(f"./.cache/lda_{hash}.npy")
    else:
        # Calculate class and overall means
        mu_class = [np.mean(data_class[i], axis=0) for i in range(len(data_class))]
        mu_all = np.mean(data, axis=0)

        # Calculate between-class scatter matrix
        s_b = np.zeros((data.shape[1], data.shape[1]))
        for i in range(len(mu_class)):
            centered_data = (mu_class[i] - mu_all).reshape(1, data.shape[1])
            s_b += num_labels[i] * centered_data.T @ centered_data

        # Calculate within-class scatter matrix
        z_i = np.zeros((data.shape[0], data.shape[1]))
        for i in range(len(data_class)):
            z_i[labels_idx[i]] = data_class[i] - mu_class[i]
        s = z_i.T @ z_i
        del z_i
        del mu_class

        # Compute eigenvectors
        # s_inv = la.pinvh(s)
        # eig_vals, eig_vecs = la.eig(s_inv @ s_b)
        s += np.finfo(np.float32).eps * np.eye(s.shape[0])
        s_b += np.finfo(np.float32).eps * np.eye(s_b.shape[0])
        eig_vals, eig_vecs = la.eigh(s_b, s)
        del s_b
        del s
        # del s_inv

        # Sort eigenvectors by eigenvalues in descending order
        idx = eig_vals.argsort()[::-1]
        eig_vecs = eig_vecs[:, idx]
        eig_vecs = np.real(eig_vecs)

        # Cache eigenvectors
        if not os.path.exists("./.cache"):
            os.makedirs("./.cache")
        np.save(f"./.cache/lda_{hash}.npy", eig_vecs)

    # Return the largest n - 1 eigenvectors where n is the number of classes
    eigc = len(num_labels) - 1 if eigc is None else min(eigc, len(eig_vecs))
    return eig_vecs[:, :eigc]

## Run the Algorithm

In [ ]:
proj_matrix = lda(training_data, training_labels)

## Evaluate the Algorithm

In [ ]:
evaluate_model(proj_matrix, training_data, training_labels, testing_data, testing_labels)

# Face vs Non-Face Classification

## Prepare the Data

In [ ]:
# Load both face and non-face data
face_data = load_data('./archive')[0]
non_face_data = load_data('./non_face')[0]

# Randomize the data to avoid bias when splitting
np.random.seed(43)
np.random.shuffle(face_data)
np.random.seed(43)
np.random.shuffle(non_face_data)

# Equate the number of samples for each class
num_samples = min(face_data.shape[0], non_face_data.shape[0])
face_data = face_data[:num_samples]
non_face_data = non_face_data[:num_samples]

# Concatenate the data and create labels
data = np.concatenate((face_data, non_face_data))
labels = np.concatenate((np.ones(face_data.shape[0]), np.zeros(non_face_data.shape[0])))

# Free up memory
del face_data
del non_face_data

# Split the data into training and testing sets with a 1:1 ratio
training_data = data[1::2]
training_labels = labels[1::2]
testing_data = data[::2]
testing_labels = labels[::2]

## Perform LDA (using **one** eigenvector)

In [ ]:
proj_matrix = lda(training_data, training_labels, 1)

## Test the Accuracy of the Model

In [ ]:
evaluate_model(proj_matrix, training_data, training_labels, testing_data, testing_labels)

## How Many Dominant Eigenvectors to Use

LDA seeks to maximize the separation between the classes in a lower dimensionality, hence the number of dominant eigenvectors should correspond to the minimum of (number of classes and the original dimensionality), which means in this case one eigenvector could be enough.

We can still try using 20, 41 dominant eigenvectors to see if the accuracy improves.

In [ ]:
def compare_egc(training_data, training_labels, testing_data, testing_labels, egcs):
    """
    Compares the accuracy of the LDA model for different numbers of eigenvectors.

    Args:
        training_data: A matrix of shape (num_training_samples, num_features) containing the training data.
        training_labels: A vector of shape (num_training_samples,) containing the labels of the training data.
        testing_data: A matrix of shape (num_testing_samples, num_features) containing the testing data.
        testing_labels: A vector of shape (num_testing_samples,) containing the labels of the testing data.
        egcs: A list of integers containing the number of eigenvectors to use for the projection matrix.

    Returns:
        None
    """
    # Initialize the list of accuracies
    accuracies = [0] * len(egcs)

    # Loop through the list of eigenvector counts
    for i, egc in enumerate(egcs):
        # Compute the projection matrix
        proj_matrix = lda(training_data, training_labels, egc)

        # Evaluate the model
        print(f"Evaluating model with {egc} eigenvectors:")
        accuracies[i] = evaluate_model(proj_matrix, training_data, training_labels, testing_data, testing_labels)

    # Plot the results
    plt.plot(egcs, accuracies)
    plt.xlabel("Number of Eigenvectors")
    plt.ylabel("Accuracy")
    plt.title("Accuracy vs. Number of Eigenvectors")
    plt.show()

In [ ]:
# Compare the accuracy of the LDA model for different numbers of eigenvectors
compare_egc(training_data, training_labels, testing_data, testing_labels, [1, 21, 40])

## Effect of the Number of Non-Face Images

In [ ]:
def compare_nf_ratio(training_data, training_labels, testing_data, testing_labels, ratios):
    """
    Compares the accuracy of the LDA model for different ratios of non-face to face images.

    Args:
        training_data: A matrix of shape (num_training_samples, num_features) containing the training data.
        training_labels: A vector of shape (num_training_samples,) containing the labels of the training data.
        testing_data: A matrix of shape (num_testing_samples, num_features) containing the testing data.
        testing_labels: A vector of shape (num_testing_samples,) containing the labels of the testing data.
        ratios: A list of floats containing the ratios of non-face to face images to use.

    Returns:
        None
    """
    # Initialize the list of accuracies
    accuracies = [0] * len(ratios)

    # Loop through the list of ratios
    for i, ratio in enumerate(sorted(ratios, reverse=True)):
        # Reduce the number of non-face images to ratio * num_face_images
        num_face_images = np.sum(training_labels == 1)
        num_non_face_images = np.sum(training_labels == 0)
        reduced_amount = num_non_face_images - int(ratio * num_face_images)
        training_data = training_data[:-reduced_amount]
        training_labels = training_labels[:-reduced_amount]

        # Compute the projection matrix
        proj_matrix = lda(training_data, training_labels, 1)

        # Evaluate the model
        print(f"Evaluating model with {ratio} ratio of non-face to face images:")
        accuracies[i] = evaluate_model(proj_matrix, training_data, training_labels, testing_data, testing_labels)

    # Plot the results
    plt.plot(ratios, accuracies)
    plt.xlabel("Ratio of Non-Face to Face Images")
    plt.ylabel("Accuracy")
    plt.title("Accuracy vs. Ratio of Non-Face to Face Images")
    plt.show()

In [ ]:
# Compare the accuracy of the LDA model for different ratios of non-face to face images
compare_nf_ratio(training_data, training_labels, testing_data, testing_labels, [0.3, 0.6, 1.0])

# Effect of Splitting Ratio (Bonus)

## Prepare the Data

In [ ]:
# Load the data
data, labels = load_data('./archive')

# Get the indices of the training and testing data
# Data will be splitted into training and testing sets with a 7:3 ratio
training_indices = np.arange(len(data)) % 10 < 7
testing_indices = np.arange(len(data)) % 10 >= 7

# Split the data
training_data = data[training_indices]
training_labels = labels[training_indices]
testing_data = data[testing_indices]
testing_labels = labels[testing_indices]

## Plot a Random Sample From the Dataset

In [ ]:
print("A sample of images from the training set:")
plot_images_samples(training_data, training_labels, 7)

print("A sample of images from the testing set:")
plot_images_samples(testing_data, testing_labels, 3)

## Train the Model

In [ ]:
# Initialize the list of alpha values
alphas = [0.8, 0.85, 0.9, 0.95]

# Compute the projection matrices for each alpha
proj_matrices = pca(training_data, alphas)

## Evaluate the Model

In [ ]:
# Evaluate the performance of the k-NN classifier for each alpha
for i in range(len(alphas)):
  print(f"Evaluating model with alpha = {alphas[i]}:")
  evaluate_model(proj_matrices[i], training_data, training_labels, testing_data, testing_labels)

# Compare with Sklearn PCA and LDA (Bonus)

## Prepare the Data

In [ ]:
# Load the data
data, labels = load_data('./archive')

# Split the data into training and testing sets with a 1:1 ratio
training_data = data[1::2]
training_labels = labels[1::2]
testing_data = data[::2]
testing_labels = labels[::2]

## Compare PCA

In [ ]:
def compare_pca(training_data, training_labels, testing_data, testing_labels, alphas):
    """
    Compares the accuracy of Sklearn's PCA algorithm with that of our implementation.

    Args:
        training_data: A matrix of shape (num_training_samples, num_features) containing the training data.
        training_labels: A vector of shape (num_training_samples,) containing the labels of the training data.
        testing_data: A matrix of shape (num_testing_samples, num_features) containing the testing data.
        testing_labels: A vector of shape (num_testing_samples,) containing the labels of the testing data.
        alphas: A list of floats containing the alpha values to use.

    Returns:
        None
    """

    # Initialize the list of accuracies
    sklearn_accuracies = [0] * len(alphas)
    our_accuracies = [0] * len(alphas)

    # Loop through the list of alpha values
    for i in range(len(alphas)):
        # Predict the labels using Sklearn's PCA
        sklearn_pca = SKLPCA(n_components=alphas[i])
        proj_training_data = sklearn_pca.fit_transform(training_data)
        proj_testing_data = sklearn_pca.transform(testing_data)
        knn = KNeighborsClassifier(n_neighbors=1)
        knn.fit(proj_training_data, training_labels)
        y_pred = knn.predict(proj_testing_data)
        sklearn_accuracies[i] = np.mean(y_pred == testing_labels)

        # Predict the labels using our implementation of PCA
        proj_matrix = pca(training_data, [alphas[i]])[0]
        proj_training_data = np.dot(training_data, proj_matrix)
        proj_testing_data = np.dot(testing_data, proj_matrix)
        knn = KNeighborsClassifier(n_neighbors=1)
        knn.fit(proj_training_data, training_labels)
        y_pred = knn.predict(proj_testing_data)
        our_accuracies[i] = np.mean(y_pred == testing_labels)

        # Print the results
        print(f"Evaluating model with alpha = {alphas[i]}:")
        print(f"Sklearn's PCA accuracy: {sklearn_accuracies[i]}")
        print(f"Our PCA accuracy: {our_accuracies[i]}")

    # Plot the results
    plt.plot(alphas, sklearn_accuracies, label="Sklearn's PCA")
    plt.plot(alphas, our_accuracies, label="Our PCA")
    plt.xlabel("Alpha")
    plt.ylabel("Accuracy")
    plt.title("Accuracy vs. Alpha")
    plt.legend()
    plt.show()



In [ ]:
compare_pca(training_data, training_labels, testing_data, testing_labels, [0.8, 0.85, 0.9, 0.95])